In [1]:
import numpy as np
import pandas as pd
import plotly.io as pio
pio.templates.default = "simple_white"

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# load the data
data = pd.read_csv("C:\\Users\omerl\OneDrive\Desktop\iml_repo\IML.HUJI\datasets\\agoda_cancellation_train.csv",engine='python')
data_frame = pd.DataFrame(data)

In [ ]:
# get dummies for the categorical charge options
data_frame['charge_option'] = data_frame['charge_option'].apply(lambda x: 1 if x == 'Pay Now' else 0)

In [ ]:
# fill missing values at cancellation_datetime column with zero values
# data_frame['cancellation_datetime'] = data_frame['cancellation_datetime'].fillna(0)
# data_frame['cancellation_datetime'] = (
#     pd.to_numeric(data_frame['cancellation_datetime'],errors='coerce').fillna(1))
# data_frame=data_frame.rename(columns = {'cancellation_datetime':'is_cancelled'})


In [ ]:
# cleaning the data remove missing values and duplicates
data_frame['cancellation_datetime'] = data_frame['cancellation_datetime'].fillna('2022-02-01')
data_frame['cancellation_datetime'] = pd.to_datetime(data_frame['cancellation_datetime'])
filtered_df = data_frame.loc[(data_frame['cancellation_datetime'] <= '2018-09-29') | (data_frame['cancellation_datetime'] == '2022-02-01')]
filtered_df['booking_datetime'] = filtered_df['booking_datetime'].apply(pd.to_datetime)
filtered_df['is_cancelled'] = (filtered_df['cancellation_datetime'] - filtered_df['booking_datetime']).dt.days
filtered_df['is_cancelled'] = filtered_df['is_cancelled'].apply(lambda x: 1 if x<=44 and x >=35 else 0)
data_frame = filtered_df

In [ ]:
# drop columns that are not relevant to the prediction
data_frame = data_frame.drop(['h_customer_id', 'h_booking_id', 'hotel_area_code'], axis=1)
data_frame = data_frame.drop(['original_payment_method', 'language','original_payment_currency'], axis=1)

In [ ]:
# fill missing values with 0 at the following columns
data_frame['request_nonesmoke'] = data_frame['request_nonesmoke'].fillna(0)
data_frame['request_earlycheckin'] = data_frame['request_earlycheckin'].fillna(0)
data_frame['request_latecheckin'] = data_frame['request_latecheckin'].fillna(0)
data_frame['request_twinbeds'] = data_frame['request_twinbeds'].fillna(0)
data_frame['request_largebed'] = data_frame['request_largebed'].fillna(0)
data_frame['request_airport'] = data_frame['request_airport'].fillna(0)
data_frame['request_highfloor'] = data_frame['request_highfloor'].fillna(0)
data_frame['no_of_extra_bed'] = data_frame['no_of_extra_bed'].apply(lambda x: 1 if x>=1 else 0)
data_frame = data_frame.rename(columns = {'no_of_extra_bed':'request_extra_bed'})
### merge all request_* columns into one column
data_frame['request_twinbeds'] = data_frame['request_largebed'] + data_frame['request_twinbeds'] + data_frame['request_airport'] + data_frame['request_highfloor'] + data_frame['request_earlycheckin'] + data_frame['request_latecheckin'] + data_frame['request_nonesmoke']
# rename the request_twinbeds column to number_of_requests
data_frame = data_frame.rename(columns = {'request_twinbeds':'number_of_requests'})
data_frame['number_of_requests'] = data_frame['number_of_requests'].apply(lambda x: 0 if x < 1 else 1)
data_frame.drop(['request_largebed', 'request_airport', 'request_highfloor', 'request_earlycheckin', 'request_latecheckin', 'request_nonesmoke'], axis=1, inplace=True)
# print the correlation between the features and the is_cancelled column

In [ ]:
delete_columns = []
dummy_2 = pd.get_dummies(data_frame['cancellation_policy_code'],prefix="cancel_policy")
dummy_2['is_cancelled'] = data_frame['is_cancelled']
print(dummy_2.corr()['is_cancelled'].sort_values(ascending=False)[:80])
print(dummy_2.corr()['is_cancelled'].sort_values(ascending=True)[:20])
for col in dummy_2:
    if(dummy_2[col].sum() < 200):
        dummy_2.drop(col, inplace=True, axis=1)
for col in dummy_2:
    if(col == 'is_cancelled'):
        continue
    if(np.abs(dummy_2['is_cancelled'].corr(dummy_2[col])) < 0.005):
        dummy_2.drop(col, inplace=True, axis=1)
        delete_columns.append(col)
print(delete_columns)
#data_frame = pd.concat([data_frame, dummy], axis=1)
# print the correlation between the features and the is_cancelled column
dummy_2.drop('is_cancelled', inplace=True, axis=1)
# concat the dummies to the data_frame
data_frame = pd.concat([data_frame, dummy_2], axis=1)

In [ ]:

# # fill missing values of the hotel_chain_code column with "Unknown"
# data_frame['hotel_chain_code'] = data_frame['hotel_chain_code'].fillna("Unknown")
# # fill missing values of the hotel_brand_code column with "Unknown"
# data_frame['hotel_brand_code'] = data_frame['hotel_brand_code'].fillna("Unknown")
# dummy = pd.get_dummies(data_frame['hotel_chain_code'], prefix='hbc_')
# for col in dummy:
#     if(dummy[col].sum() < 200):
#         dummy.drop(col, inplace=True, axis=1)
# # conect the dummies to the data frame
# data_frame = pd.concat([data_frame, dummy], axis=1)
# # print the correlation between the features and the is_cancelled column
# data_frame.corr()['is_cancelled'].sort_values(ascending=False)

In [ ]:
# explore the data in the Booking_datetime column
# and the checkin_date and checkout_date columns
# calculate the difference in days between the Booking_datetime and the checkin_date
# and the difference in days between the Booking_datetime and the checkout_date

data_frame['checkin_date'] = data_frame['checkin_date'].apply(pd.to_datetime)
data_frame['checkout_date'] = data_frame['checkout_date'].apply(pd.to_datetime)
data_frame['booking_datetime'] = data_frame['booking_datetime'].apply(pd.to_datetime)
# insert new column with the difference in days between the Booking_datetime and the checkin_date
data_frame['Lead_day'] = (data_frame['checkin_date'] - data_frame['booking_datetime']).dt.days
data_frame['stay_len'] = (data_frame['checkout_date'] - data_frame['checkin_date']).dt.days
# replace all the negative values with 0
data_frame['Lead_day'] = data_frame['Lead_day'].apply(lambda x: 0 if x < 0 else x)
data_frame['stay_len'] = data_frame['stay_len'].apply(lambda x: 0 if x < 0 else x)

In [ ]:
data_frame['original_payment_type'] = data_frame['original_payment_type'].apply(lambda x: 0 if x == 'Credit Card' else 1)

In [ ]:
dummy = pd.get_dummies(data_frame['origin_country_code'], prefix="origin_country_code")
dummy.insert(0, 'is_cancelled',data_frame['is_cancelled'])
dummy.fillna("none")
print(dummy.corr()['is_cancelled'].sort_values(ascending=False)[:80])
print(dummy.corr()['is_cancelled'].sort_values(ascending=True)[:20])
counter = 0
for col in dummy:
    if(dummy[col].sum() < 200):
        dummy.drop(col, inplace=True, axis=1)
        delete_columns.append(col)
for col in dummy:
    if(col == 'is_cancelled'):
        continue
    if(np.abs(dummy['is_cancelled'].corr(dummy[col])) < 0.005):
        dummy.drop(col, inplace=True, axis=1)
        delete_columns.append(col)
print(delete_columns)
dummy.drop('is_cancelled', inplace=True, axis=1)
data_frame = pd.concat([data_frame, dummy], axis=1)

In [ ]:
dummy_2 = pd.get_dummies(data_frame['guest_nationality_country_name'], prefix="guest_national")
dummy_2['is_cancelled'] = data_frame['is_cancelled']
print(dummy_2.corr()['is_cancelled'].sort_values(ascending=False)[:80])
print(dummy_2.corr()['is_cancelled'].sort_values(ascending=True)[:20])
for col in dummy_2:
    if(dummy_2[col].sum() < 100):
        dummy_2.drop(col, inplace=True, axis=1)
        delete_columns.append(col)
for col in dummy_2:
    if(col == 'is_cancelled'):
        continue
    if(np.abs(dummy_2['is_cancelled'].corr(dummy_2[col])) < 0.005):
        dummy_2.drop(col, inplace=True, axis=1)
        delete_columns.append(col)
#data_frame = pd.concat([data_frame, dummy], axis=1)
# print the correlation between the features and the is_cancelled column
dummy_2.drop('is_cancelled', inplace=True, axis=1)

In [ ]:
# concat the dummies to the data frame
data_frame = pd.concat([data_frame, dummy_2], axis=1)
# data_frame.drop(['guest_nationality_country_name', 'customer_natio'], axis=1, inlpace=True)

In [ ]:
print(data_frame.columns)
data_frame.drop(['cancellation_policy_code', 'checkout_date', 'origin_country_code','guest_nationality_country_name','hotel_id','hotel_live_date', 'hotel_country_code','hotel_live_date','customer_nationality', 'guest_nationality_country_name',
                 'origin_country_code'], axis=1, inplace=True)

In [ ]:
dummy_3 = pd.get_dummies(data_frame['accommadation_type_name'], prefix="type_hotel")
dummy_3['is_cancelled'] = data_frame['is_cancelled']
print(dummy_3.columns)
print(dummy_2.corr()['is_cancelled'].sort_values(ascending=False)[:80])
print(dummy_2.corr()['is_cancelled'].sort_values(ascending=True)[:20])
for col in dummy_3:
    if(dummy_3[col].sum() < 100):
        dummy_3.drop(col, inplace=True, axis=1)
        delete_columns.append(col)
for col in dummy_3:
    if(col == 'is_cancelled'):
        continue
    if(np.abs(dummy_3['is_cancelled'].corr(dummy_3[col])) < 0.005):
        dummy_3.drop(col, inplace=True, axis=1)
        delete_columns.append(col)

#data_frame = pd.concat([data_frame, dummy], axis=1)
# print the correlation between the features and the is_cancelled column
dummy_3.drop('is_cancelled', inplace=True, axis=1)


In [ ]:
data_frame = pd.concat([data_frame, dummy_3], axis=1)
# print the correlation between the features and the is_cancelled column
print(data_frame.columns)
print(data_frame.info())
print(data_frame.isnull().sum())


In [ ]:
data_frame.drop(['accommadation_type_name'], axis=1, inplace=True)

In [ ]:
print(delete_columns)
# write delete columns to a file
with open('delete_columns.txt', 'w') as f:
    f.write(str(delete_columns))

In [ ]:
print(data_frame.columns)
print(data_frame.info())
print(data_frame.shape)